In [1]:
import numpy as np 

# NEAT
1. Classes and Functions
- 1.1. Neural Network (Genotype)->Phenotype, input, output dim, contains mutation: 
- 1.2. Genotype: A->B: connection gene, A:Node gene, is_disabled, weight, keep track of gene history
- 1.3. Crossover (Genotype1, Genotype2)->Genotype12
- 1.4. Species, represented by random member
- 1.5. Speciation (List of Species)-> List of Species
- 1.6. Fitness Calculation (Species)
- 1.7.

In [ ]:
# Evolver:
n_networks = 150

# Fitness:
c1 = 1.0
c2 = 1.0
c3 = 0.4
dt = 3 

# Mutation
mutate_weight_prob = 0.8
mutate_weight_perturb = 0.9
mutate_weight_random = 1 - mutate_weight_perturb
mutate_add_node_prob = 0.03
mutate_add_node_prob_large_pop = 0.3
mutate_add_link_prob = 0.05

offspring_without_crossover = 0.25
interspecies_mate_rate = 0.001

def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-4.9*x))


In [53]:

class History:
    def __init__(self):
        self.last_node_id = 0
        self.last_connection_id = 0
        self.node_innovations = {}
        self.connection_innovations = {}
    
    def add_node_gene(self, start_node_id, end_node_id): # node is added between start and end
        self.last_node_id += 1
        



class Connection_Gene_History:
    def __init__(self):
        self.innovation_number = 0
        self.history = {}
    
    def get_innovation_number(self, connection):
        if connection not in self.history:
            self.innovation_number += 1
            self.history[connection] = self.innovation_number
        
        return self.history[connection]
    
    def __contains__(self, connection):
        return connection in self.history

class Node_Gene_History:
    def __init__(self):
        self.innovation_number = 0
        self.history = {}
        self.node_levels = {}
    
    def get_innovation_number(self, connection, src_node):
        
        if connection not in self.history:
            self.innovation_number += 1
            self.history[connection] = self.innovation_number
            self.node_level[self.innovation_number] = self.node_level[src_node] + 1
        
        return self.history[connection], self.node_levels[self.innovation_number]
    
    def __contains__(self, connection):
        return connection in self.history



class Node_Gene:
    def __init__(self, src_node, dst_node, node_gene_history:Node_Gene_History):
        connection = str(src_node)+'->'+str(dst_node)
        self.innovation_number, self.node_level = node_gene_history.get_innovation_number(connection)
            
        self.src_node = src_node
        self.dst_node = dst_node

class Connection_Gene:
    def __init__(self, in_node, out_node, weight, is_disabled, connection_gene_history:Connection_Gene_History):
        connection = str(in_node)+'->'+str(out_node)
        self.in_node = in_node
        self.out_node = out_node
        self.weight = weight
        self.is_disabled = is_disabled
        self.innovation_number = connection_gene_history.get_innovation_number(connection)
        
            
class Genotype:
    def __init__(self, node_genes, connection_genes,
                 node_gene_history:Node_Gene_History, connection_gene_history:Connection_Gene_History,
                 mutate_weight_prob, mutate_weight_perturb, mutate_weight_random, mutate_add_node_prob, mutate_add_node_prob_large_pop, mutate_add_link_prob):
        
        self.node_genes = node_genes
        self.connection_genes = connection_genes
        self.node_gene_history = node_gene_history
        self.connection_gene_history = connection_gene_history
        self.mutate_weight_prob = mutate_weight_prob
        self.mutate_weight_perturb = mutate_weight_perturb
        self.mutate_weight_random = mutate_weight_random
        self.mutate_add_node_prob = mutate_add_node_prob
        self.mutate_add_node_prob_large_pop = mutate_add_node_prob_large_pop
        self.mutate_add_link_prob = mutate_add_link_prob
        
    
    def mutate(self):
        # mutate weight
        for connection_gene in self.connection_genes:
            if np.random.rand() < self.mutate_weight_prob:
                if np.random.rand() < self.mutate_weight_perturb:
                    connection_gene.weight += np.random.normal()
                else:
                    connection_gene.weight = np.random.normal()
        
        # mutate add node
        if np.random.rand() < self.mutate_add_node_prob:
            self.add_node()

        # mutate add link
        if np.random.rand() < self.mutate_add_link_prob:
            self.add_connection()
            
    def add_node(self):
        # select a random connection gene
        while True:
            connection_gene = np.random.choice(self.connection_genes)
            if not connection_gene.is_disabled:
                break 
            
        connection_gene.is_disabled = True
        
        # add node gene
        node_gene = Node_Gene(connection_gene.in_node, connection_gene.out_node, self.node_gene_history)
        self.node_genes.append(node_gene)
        
        # add connection genes, first weight is 1.0, second is the one of the original
        connection_gene1 = Connection_Gene(connection_gene.in_node, node_gene.src_node, 1.0, False, self.connection_gene_history)
        connection_gene2 = Connection_Gene(node_gene.dst_node, connection_gene.out_node, connection_gene.weight, False, self.connection_gene_history)
        self.connection_genes.append(connection_gene1)
        self.connection_genes.append(connection_gene2)
    
    def add_connection(self):
        while True:
            # first select random source node
            src = np.random.choice(self.node_genes)
            
            # select destination node where level is higher than source
            dsts = [node_gene for node_gene in self.node_genes if node_gene.node_level > src.node_level]
            if len(dsts) == 0: # case when no more connection available
                continue
            dst = np.random.choice(dsts)
            
            
            # check if connection already exists
            connection = str(src.innovation_number)+'->'+str(dst.innovation_number)
            if not connection in self.connection_gene_history:
                break
        
        # add connection gene
        connection_gene = Connection_Gene(src.innovation_number, dst.innovation_number, np.random.normal(), False, self.connection_gene_history)
        self.connection_genes.append(connection_gene)
    
    def crossover(self, other):
        
        # perform crossover, take all connection genes from the fitter parent
        # 
        
        return Genotype(node_genes, connection_genes, self.node_gene_history, self.connection_gene_history, self.mutate_weight_prob, self.mutate_weight_perturb, self.mutate_weight_random, self.mutate_add_node_prob, self.mutate_add_node_prob_large_pop, self.mutate_add_link_prob)
    
    def distance(self, other):
        pass
    
    def __str__(self):
        return str(self.node_genes) + '\n' + str(self.connection_genes)

In [54]:
node_gene_history = Node_Gene_History()
connection_gene_history = Connection_Gene_History()


In [55]:
node_gene_history.history

{}

In [52]:
a = Node_Gene(1, 3, node_gene_history)
a.innovation_number

2

In [ ]:
class NeuralNetwork():
    def __init__(self, genotype:Genotype) -> None:
        pass